In [ ]:
import time
import sys

sys.path.append('/home/cjw/Code/Unet')
sys.path.append('/media/cjw/PythonLib/Unet')
from skimage.io import imread
from matplotlib import pyplot as plt
import numpy as np

import unet
%matplotlib inline

In [ ]:
import tensorflow as tf

In [ ]:
params = dict()

params['width'] = 256
params['height'] = 256
params['depth'] = 96
params['nchannels'] = 1
params['channels'] = [1]
params['nepochs'] = [1]
params['batchsize'] = [8]
params['learning_rate'] = 0.001
params['restore'] = False
params['latent_size'] = 1
params['enc_sizes'] = [[32,3], [64, 3], [128, 3], [192, 3]]
params['dec_sizes'] = [[128,3], [64, 3], [32, 3], [params['nchannels'], 3]]
params['droprate'] = 0.25
params['stdev'] = 0.08

In [ ]:

tf.reset_default_graph()
u = unet.unet3d(params)


In [ ]:
#u.read_images('/ssd1/cjw/Data/Nuclei.tif')
u.read_mm('/ssd1/cjw/Data/nuclei.mm', shape=(720, 256, 256, 96, 2))
#u.x = u.x[:,:,:,:,:]
u.x.shape

In [ ]:
#a = u.get_batch(4, augment=True)
#ab = u.augment_batch(a)
#ab.shape

In [ ]:
# from scipy.ndimage import rotate
# # angle = 90*np.random.randint(0,4)
# # print(angle)
# # s = a[2]
# #ar = rotate(a, 270, (1,2), reshape=False, order=1)
# plt.subplot(2,1,1)
# plt.imshow(a[1,:,:,47,0])
# plt.subplot(2,1,2)
# plt.imshow(a[1,:,:,47,1])

In [ ]:

images = tf.placeholder(tf.float32, (None, 256, 256, 96, params['nchannels']))
masks = tf.placeholder(tf.float32, (None, 256, 256, 96, params['nchannels']))
learning_rate = tf.placeholder(tf.float32, ())
u.learning_rate = learning_rate

In [ ]:
enc = u.create_encoder(images, True)

In [ ]:
d = u.create_decoder()
u.create_loss(masks)
u.create_opt()

In [ ]:
u.set_validation(20)

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
u.stdev = 0.04
#u.learning_rate = 0.0001

In [ ]:
vx = np.expand_dims(u.xvalid[:,:,:,:,0], -1)
vm = np.expand_dims(u.xvalid[:,:,:,:,1], -1) 

rate = 0.001
for i in range(20000):
    t1 = time.time()
    rate = np.maximum(rate*.9992, 0.000001)
    batch = u.get_batch(8, augment=False)
    bx = np.expand_dims(batch[:,:,:,:,0], -1)
    bm = np.expand_dims(batch[:,:,:,:,1], -1)
    _, res, err = sess.run([u.opt, u.decoder_sigmoid, u.loss],
                           feed_dict={images:bx, masks:bm, learning_rate:rate})
    if i % 40 == 0:
        vres, verr = sess.run([u.decoder_sigmoid, u.loss],
                               feed_dict={images:vx, masks:vm})

        zm = vres[0,:,:,:,0].argmax(axis=(-1))
        print(err, verr, vres[0,:,:,:,0].min(), vres[0,:,:,:,0].max())
        #print(zm)
    
        plt.figure(figsize=(12,2))
        plt.subplot(1,6,1)
        plt.imshow(res[0,:,:,32,0])
        plt.subplot(1,6,2)
        plt.imshow(bm[0,:,:,32,0])
        plt.subplot(1,6,3)
        plt.imshow(bx[0,:,:,32,0])
        plt.subplot(1,6,4)
        plt.imshow(vres[0,:,:,32,0])
        plt.subplot(1,6,5)
        plt.imshow(vm[0,:,:,32,0])
        plt.subplot(1,6,6)
        plt.imshow(vx[0,:,:,32,0])
        plt.show()
    t2 = time.time()
    if i % 4 == 0:
        print(i, err, (t2 - t1), rate)
        #print(zm)

In [ ]:
z = 56
i = 5
a  = vres[i, :,:, z,0]
a = np.where(a > .1, a*0 + 1, a*0)
a = (a - a.min())/(a.max() - a.min())
a = 255*a
print(a.sum())
a = a.astype(np.int32)
b  = vm[i, :,:, z,0]
b = (b - b.min())/(b.max() - b.min())
b = 255*b
b = b.astype(np.int32)

c = vx[i,:,:,z,0]
c = (c - c.min())/(c.max() - c.min())
c = 255*c
c = c.astype(np.int32)
print(c.max(), c.min())
sh = np.stack([a, b, c], axis=2)
sh.shape

In [ ]:
plt.imshow(sh)

In [ ]:
print(vx.shape, vm.shape, vres.shape)

xo = np.stack((vx, vm, vres), 4)
xo = np.squeeze(xo)
xo = np.rollaxis(xo, 3, 1)
xo.shape

In [ ]:
smm = np.memmap('/media/cjw/Data/c3d0.mm', mode='w+', dtype=np.float32
                , shape=xo.shape)

print(type(smm))
smm[:] = xo[:]
print(type(smm))
smm.flush()
del smm

In [ ]:
import tifffile

In [ ]:
tifffile.imwrite('/ssd1/cjw/Data/w1.tif', xo)

In [ ]:
u.create_loss(masks)
ss = sess.run(u.loss, feed_dict={images:yy, masks:bb})

In [ ]:
ss.shape
plt.imshow(ss[0,:,:,44,0])

In [ ]:
rt = y[0,:,:,:,:]
from scipy.ndimage import rotate

res = rotate(rt, 45, (0, 1), order=2)

In [ ]:
plt.imshow(res[:,:,44,0])

In [ ]:
u.x.shape
u.used = None

In [ ]:
import pickle
from scipy.ndimage import rotate


def augment(b, da = 1):
    mm = np.memmap('/ssd1/cjw/Data/nuclei.mm', dtype=np.float32,
               mode='r+', shape=(720, 256, 256, 96, 2))
    nr = b.shape[0]
    d = dict()
    counter = 0
    for i in range(nr):
        print(i)
        s = b[i]  # this loses the first dimension
        mm[counter] = s
        counter += 1
        print(counter)
        angle = 0
        while angle < 360:
            angle += da
            mm[counter] = rotate(s, angle, (0,1), reshape=False, order=1)
            counter += 1
            print(counter)
            
        mm.flush()
    del mm
        
augment(u.x, 18)

In [ ]:
mm = np.memmap('/media/cjw/Data/nuclei.mm', dtype=np.float32,
               mode='r', shape=(720, 256, 256, 96, 2))

In [ ]:
plt.imshow(mm[-19,:,:,34,0])

In [ ]:
rf = .1
for i in range(20000):
    rf = rf*.9996
    
print(rf)